In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Polycystic_Kidney_Disease"
cohort = "GSE74453"

# Input paths
in_trait_dir = "../../input/GEO/Polycystic_Kidney_Disease"
in_cohort_dir = "../../input/GEO/Polycystic_Kidney_Disease/GSE74453"

# Output paths
out_data_file = "../../output/preprocess/Polycystic_Kidney_Disease/GSE74453.csv"
out_gene_data_file = "../../output/preprocess/Polycystic_Kidney_Disease/gene_data/GSE74453.csv"
out_clinical_data_file = "../../output/preprocess/Polycystic_Kidney_Disease/clinical_data/GSE74453.csv"
json_path = "../../output/preprocess/Polycystic_Kidney_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Identification of a novel risk factor for intracranial aneurysms in ADPKD using iPSC models"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['subject/sample source id: TIG114 4F1', 'subject/sample source id: TIG118 4F1', 'subject/sample source id: TIG119 4F1', 'subject/sample source id: TIG121 4F4', 'subject/sample source id: TIG975E4', 'subject/sample source id: 585A1', 'subject/sample source id: 585B1', 'subject/sample source id: TIG107 4F1', 'subject/sample source id: TIG120 4F1', 'subject/sample source id: TIG120 3F7', 'subject/sample source id: TIG975E2', 'subject/sample source id: CiRA00004', 'subject/sample source id: CiRA00005', 'subject/sample source id: CiRA00006', 'subject/sample source id: CiRA00007', 'subject/sample source id: CiRA00008', 'subject/sample source id: CiRA00009', 'subject/sample source id: CiRA00010']

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset is about iPSC models derived from ADPKD patients
# This might contain gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Trait: ADPKD status is available in row 1, but it appears all samples are from ADPKD patients
# This means the trait value is constant across all samples, making it not usable for association studies
trait_row = None  # Setting to None since it's a constant feature

# Age: There doesn't appear to be age information in the sample characteristics
age_row = None

# Gender: Available in row 2
gender_row = 2

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert trait value to binary (1 for ADPKD)"""
    # Not used since trait data is effectively not available (constant)
    return None

def convert_age(value):
    """Convert age value to continuous"""
    # Not used since age data is not available
    return None

def convert_gender(value):
    """Convert gender value to binary (0 for female, 1 for male)"""
    if ':' in value:
        gender = value.split(':', 1)[1].strip().lower()
        if 'female' in gender:
            return 0
        elif 'male' in gender:
            return 1
    return None

# 3. Save Metadata
# Conduct initial filtering and save information
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Create DataFrame from the sample characteristics dictionary
    sample_chars = {0: ['subject/sample source id: CiRA00004', 'subject/sample source id: CiRA00005', 'subject/sample source id: CiRA00006', 'subject/sample source id: CiRA00007', 'subject/sample source id: CiRA00008', 'subject/sample source id: CiRA00009', 'subject/sample source id: CiRA00010'], 
                   1: ['subject status: autosomal dominant polycystic kidney disease (ADPKD) patient'], 
                   2: ['gender: Female', 'gender: Male'], 
                   3: ['cell type: iPSC derived endothelial cells', 'cell type: iPSC derived vascular smooth muscle cells']}
    
    # Convert sample_chars to DataFrame format expected by geo_select_clinical_features
    clinical_data = pd.DataFrame(sample_chars)
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(selected_clinical_df)
    print("Preview of extracted clinical features:")
    print(preview)
    
    # Ensure output directory exists
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the extracted clinical features
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
else:
    print("Clinical data is not available or not suitable for association studies, skipping clinical feature extraction.")


Clinical data is not available or not suitable for association studies, skipping clinical feature extraction.


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107',
       '(+)E1A_r60_a135', '(+)E1A_r60_a20', '(+)E1A_r60_a22', '(+)E1A_r60_a97',
       '(+)E1A_r60_n11', '(+)E1A_r60_n9', '3xSLv1', 'A_19_P00315452',
       'A_19_P00315459', 'A_19_P00315482', 'A_19_P00315492', 'A_19_P00315493',
       'A_19_P00315502', 'A_19_P00315506', 'A_19_P00315518', 'A_19_P00315519'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Based on examining the gene identifiers in the gene expression data:
# The identifiers shown (like '7892501', '7892502', etc.) are probe IDs from a microarray platform, 
# not standard human gene symbols like BRCA1, TP53, etc.
# These probe IDs need to be mapped to their corresponding gene symbols for meaningful analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['7896736', '7896738', '7896740', '7896742', '7896744'], 'GB_LIST': [nan, nan, 'NM_001004195,NM_001005240,NM_001005484,BC136848,BC136867,BC136907,BC136908', 'NR_024437,XM_006711854,XM_006726377,XR_430662,AK298283,AL137655,BC032332,BC118988,BC122537,BC131690,NM_207366,AK301928,BC071667', 'NM_001005221,NM_001005224,NM_001005277,NM_001005504,BC137547,BC137568'], 'SPOT_ID': ['chr1:53049-54936', 'chr1:63015-63887', 'chr1:69091-70008', 'chr1:334129-334296', 'chr1:367659-368597'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': ['53049', '63015', '69091', '334129', '367659'], 'RANGE_STOP': ['54936', '63887', '70008', '334296', '368597'], 'total_probes': [7.0, 31.0, 24.0, 6.0, 36.0], 'gene_assignment': ['---', 'ENST00000328113 // OR4G2P // olfactory receptor, family 4, subfamily G, member 2 pseudogene // ---

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns to use for mapping
probe_col = 'ID'
gene_col = 'gene_assignment'

# 2. Get gene mapping dataframe by extracting the two columns from the gene annotation dataframe
gene_mapping = get_gene_mapping(gene_annotation, probe_col, gene_col)

# 3. Convert probe-level measurements to gene expression data by applying the gene mapping
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Preview the first few rows of the gene data
print("Gene expression data after mapping:")
print(gene_data.shape)
print(gene_data.index[:10])


Gene expression data after mapping:
(0, 31)
Index([], dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print(f"First few normalized gene symbols: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Create a clinical dataframe with sample IDs from gene data
sample_ids = normalized_gene_data.columns.tolist()
print(f"Number of samples from gene data: {len(sample_ids)}")
print(f"First few sample IDs: {sample_ids[:5]}")

# From the background information, we know this dataset has ccRCC samples with VHL subtypes
# The information mentions "A third minor subgroup has distinct expression profiles consistent
# with von Hippel-Lindau wild type status"
# Since we can't identify which samples are which from the clinical data, we'll set a constant
# trait value for all samples (this will be filtered out as biased in quality check)
clinical_df = pd.DataFrame(index=sample_ids)
clinical_df[trait] = 1  # Set all samples with the trait (this is likely biased)

print(f"Clinical data shape: {clinical_df.shape}")
print(clinical_df.head())

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 3. Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_df.T, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print(linked_data.head())

# 4. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Shape after handling missing values: {linked_data.shape}")

# 5. Determine whether the trait and demographic features are severely biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 6. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note=f"Dataset contains gene expression data for {len(unbiased_linked_data)} ccRCC samples, but no VHL status information could be extracted from the sample metadata."
)

# 7. Save the data if it's usable
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    # Save the data
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data quality check failed. The dataset is not suitable for association studies.")

Normalized gene data shape: (0, 31)
First few normalized gene symbols: []
Normalized gene data saved to ../../output/preprocess/Polycystic_Kidney_Disease/gene_data/GSE74453.csv
Number of samples from gene data: 31
First few sample IDs: ['GSM1920920', 'GSM1920921', 'GSM1920922', 'GSM1920923', 'GSM1920924']
Clinical data shape: (31, 1)
            Polycystic_Kidney_Disease
GSM1920920                          1
GSM1920921                          1
GSM1920922                          1
GSM1920923                          1
GSM1920924                          1
Clinical data saved to ../../output/preprocess/Polycystic_Kidney_Disease/clinical_data/GSE74453.csv
Linked data shape: (31, 1)
           Polycystic_Kidney_Disease
GSM1920920                         1
GSM1920921                         1
GSM1920922                         1
GSM1920923                         1
GSM1920924                         1
Shape after handling missing values: (0, 1)
Quartiles for 'Polycystic_Kidney_Disease':
